In [10]:
import pandas as pd
import math
file = "points.xlsx"
data = pd.read_excel(file)
def compute_player_summaries(data):
    """
    Compute player summaries based on the provided data.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A summary dataframe containing computed metrics for each player.
    """
    # Define aggregation functions for the columns of interest
    aggregations = {
        'Date': 'count',
        'Penalty': 'sum',
        'Friend Referrals': 'sum',
        'Own Goals': 'sum',
        'Goals Conceded': 'sum',
        'Goals': ['mean', 'sum'],
        'Total Points': ['mean', 'sum'],
        'MVP': 'sum',
        'SPL Bonus': 'sum'
    }
    
    # Use groupby with multiple aggregation functions
    player_summary = data.groupby('Player').agg(aggregations)
    
    # Flatten hierarchical columns
    player_summary.columns = ['_'.join(col).strip() for col in player_summary.columns.values]
    
    # Rename columns for clarity
    columns_rename = {
        'Date_count': 'Games Played',
        'Penalty_sum': 'Penalties',
        'Friend Referrals_sum': 'Friend Referrals',
        'Own Goals_sum': 'Own Goals',
        'Goals Conceded_sum': 'Goals Conceded',
        'Goals_mean': 'Average Goals/Game',
        'Goals_sum': 'Total Goals',
        'Total Points_mean': 'Average Total Points/Game',
        'Total Points_sum': 'Total Points',
        'MVP_sum': 'MVP',
        'SPL Bonus_sum': 'SPL Bonus'
    }
    player_summary.rename(columns=columns_rename, inplace=True)
    
    # Calculate games won
    games_won = data[data['Team'] == data['Winning Team']].groupby('Player').size()
    player_summary['Games Won'] = games_won
    player_summary['Games Won'].fillna(0, inplace=True)
    
    # Calculate win ratio
    player_summary['Win Ratio'] = player_summary['Games Won'] / player_summary['Games Played']
    
    return player_summary

def calculate_cumulative_points_and_rank(data):
    """
    Calculate cumulative points and rank changes for each player.
    
    Args:
    - data (pd.DataFrame): The input data containing player details and performance metrics.
    
    Returns:
    - pd.DataFrame: A dataframe containing the 'Rank Change' for the latest game for each player.
    """
    # Calculate cumulative Total Points for each player after each game
    data['Cumulative Points'] = data.groupby('Player')['Total Points'].cumsum()
    
    # Determine the player's rank based on these cumulative points after each game
    data['Rank'] = data.groupby('Date')['Cumulative Points'].rank(method="first", ascending=False)
    
    # Sort data to ensure we process in chronological order for each player
    data_sorted = data.sort_values(by=['Player', 'Date'])
    
    # Calculate the change in rank between each game for every player
    data_sorted['Rank Change'] = data_sorted.groupby('Player')['Rank'].diff().fillna(0)
    
    # Extract the latest rank change for each player
    latest_rank_change = data_sorted.groupby('Player').apply(lambda x: x.iloc[-1])['Rank Change']
    
    return latest_rank_change

# Integrate functions to generate the final summary
player_summary = compute_player_summaries(data)
latest_rank_change = calculate_cumulative_points_and_rank(data)

# Add 'Rank Change' to the summary
player_summary['Rank Change'] = latest_rank_change

# Fill NaN values with 0
player_summary = player_summary.fillna(0)

# Convert specific columns to integer type
cols_to_int = ['Games Won', 'MVP', 'SPL Bonus', 'Rank Change']
player_summary[cols_to_int] = player_summary[cols_to_int].astype(int)

# Calculate the overall rank based on the 'Total Points'
player_summary['Overall Rank'] = player_summary['Total Points'].rank(method="min", ascending=False).astype(int)

# Reorder columns to have 'Overall Rank' at the front
column_order = ['Overall Rank'] + [col for col in player_summary if col != 'Overall Rank']
final_summary = player_summary[column_order]

# Sorting the final_summary DataFrame by 'Overall Rank' in ascending order
sorted_summary = final_summary.sort_values(by='Overall Rank')

# Setting option to display all rows
pd.set_option('display.max_rows', None)

sorted_summary


,Overall Rank,Games Played,Penalties,Friend Referrals,Own Goals,Goals Conceded,Average Goals/Game,Total Goals,Average Total Points/Game,Total Points,MVP,SPL Bonus,Games Won,Win Ratio,Rank Change
Player,,,,,,,,,,,,,,,
Daniele (Francesco),1,17,0,1,0,108,3.588235,61,14.470588,246,0,0,12,0.705882,0
Luca Stoppi,2,23,1,4,2,166,0.782609,18,10.652174,245,1,1,8,0.347826,1
Andrea Limonta,3,22,0,2,1,154,0.136364,3,10.500000,231,0,1,11,0.500000,0
Cameron McAinsh,4,22,0,1,0,181,1.818182,40,10.318182,227,0,0,9,0.409091,-1
Damiano Barbanera,5,21,0,2,0,161,2.285714,48,10.571429,222,0,1,7,0.333333,-2
Giovanni Aiello,6,21,0,0,0,147,0.571429,12,10.142857,213,0,2,11,0.523810,2
Sergio,7,17,0,2,0,106,0.705882,12,10.764706,183,2,0,9,0.529412,2
Federico Paolucci,8,19,0,0,0,140,0.263158,5,9.473684,180,1,1,9,0.473684,-2
Mazzu,9,14,0,3,1,110,3.285714,46,12.642857,177,1,2,4,0.285714,-2
